In [ ]:
#from google.colab import files
import pandas as pd
import numpy as np
import calendar
# import warnings

import matplotlib.pyplot as plt

## RFF Procesado

In [ ]:
# Cargamos el archivo Excel - RFF Procesado
# --------------

ruta_path = 'C:/cesar_quezada/ProyectosIA/Proyectos_PALMAS'               # -> Casa
# ruta_path = 'C:/Users/usuario/Documents/DOCUMENTOS_CONSULTOR_5_BI'      # -> Trabajo

path_sabana_fabrica_nh = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_fabrica/53f6869d-fabrica-tea-nh.xlsx'
path_sabana_fabrica_pw = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_fabrica/53f6869d-fabrica-tea-pw.xlsx'
path_sabana_fabrica_sh = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_fabrica/53f6869d-fabrica-tea-sh.xlsx'
path_sabana_fabrica_tl = '/PROYECTO_001_analitica_descriptiva_TEA/data/external/bbdd_fabrica/53f6869d-fabrica-tea-tl.xlsx'


# Cargar la hoja 2 (indexada desde 0, por lo tanto la hoja 2 es 'sheet_name=1')
df_sabana_fabrica_nh = pd.read_excel(ruta_path + path_sabana_fabrica_nh,sheet_name='Sheet1')
df_sabana_fabrica_pw = pd.read_excel(ruta_path + path_sabana_fabrica_pw,sheet_name='Sheet1')
df_sabana_fabrica_sh = pd.read_excel(ruta_path + path_sabana_fabrica_sh,sheet_name='Sheet1')
df_sabana_fabrica_tl = pd.read_excel(ruta_path + path_sabana_fabrica_tl,sheet_name='Sheet1')

# Eliminar las filas donde cualquiera de las columnas tenga el valor 'Totales' o NaN
df_sabana_fabrica_nh = df_sabana_fabrica_nh[~df_sabana_fabrica_nh[['AÑO', 'MES', 'DIA']].isin(['Totales']).any(axis=1) & df_sabana_fabrica_nh[['AÑO', 'MES', 'DIA']].notna().all(axis=1)]
df_sabana_fabrica_pw = df_sabana_fabrica_pw[~df_sabana_fabrica_pw[['AÑO', 'MES', 'DIA']].isin(['Totales']).any(axis=1) & df_sabana_fabrica_pw[['AÑO', 'MES', 'DIA']].notna().all(axis=1)]
df_sabana_fabrica_sh = df_sabana_fabrica_sh[~df_sabana_fabrica_sh[['AÑO', 'MES', 'DIA']].isin(['Totales']).any(axis=1) & df_sabana_fabrica_sh[['AÑO', 'MES', 'DIA']].notna().all(axis=1)]
df_sabana_fabrica_tl = df_sabana_fabrica_tl[~df_sabana_fabrica_tl[['AÑO', 'MES', 'DIA']].isin(['Totales']).any(axis=1) & df_sabana_fabrica_tl[['AÑO', 'MES', 'DIA']].notna().all(axis=1)]

df_sabana_fabrica_nh['PLANTACION'] = 'Nuevo Horizonte'
df_sabana_fabrica_pw['PLANTACION'] = 'Palmawasi'
df_sabana_fabrica_sh['PLANTACION'] = 'Shanuzi'
df_sabana_fabrica_tl['PLANTACION'] = 'Tulumayo'

# Concatenar los DataFrames
df_sabana_fabrica_full = pd.concat([df_sabana_fabrica_nh, df_sabana_fabrica_pw, df_sabana_fabrica_sh, df_sabana_fabrica_tl], ignore_index=True)

print(df_sabana_fabrica_full.dtypes)
df_sabana_fabrica_full.head(10)


In [ ]:
# CREANDO VARIABLES IMPORTANTES
# --------------
# -----------

df_sabana_fabrica_full['DIA'] = df_sabana_fabrica_full['DIA'].astype('int64')

meses_a_numeros = {
    'Enero': 1,
    'Febrero': 2,
    'Marzo': 3,
    'Abril': 4,
    'Mayo': 5,
    'Junio': 6,
    'Julio': 7,
    'Agosto': 8,
    'Setiembre': 9,
    'Octubre': 10,
    'Noviembre': 11,
    'Diciembre': 12
}

# Aplicar la conversión al DataFrame
df_sabana_fabrica_full['MES_NRO'] = df_sabana_fabrica_full['MES'].map(meses_a_numeros)
# Creamos variable fecha
df_sabana_fabrica_full['FECHA'] = pd.to_datetime(df_sabana_fabrica_full['AÑO'].astype(str) + '-' + df_sabana_fabrica_full['MES_NRO'].astype(str) + '-' + df_sabana_fabrica_full['DIA'].astype(str), 
                                                 format='%Y-%m-%d', errors='coerce')

df_sabana_fabrica_full['AÑO_MES'] = df_sabana_fabrica_full['AÑO'].astype(str) + '-' + df_sabana_fabrica_full['MES_NRO'].astype(str)

print(f"{df_sabana_fabrica_full['AÑO'].isna().sum()} valores perdidos para variable año")
print(f"{df_sabana_fabrica_full['MES_NRO'].isna().sum()} valores perdidos para variable mes_nro")
print(f"{df_sabana_fabrica_full['DIA'].isna().sum()} valores perdidos para variable día")
print(df_sabana_fabrica_full.PLANTACION.unique())

# Guardamos la data en processed
# -----
# Especifica la ruta donde deseas guardar el CSV
ruta_sabana_fabrica_full =  '/PROYECTO_001_analitica_descriptiva_TEA/data/processed/df_sabana_fabrica_full.csv'  # Cambia esta ruta a la que necesites
# Guardar el DataFrame en formato CSV
df_sabana_fabrica_full.to_csv(ruta_path + ruta_sabana_fabrica_full, index=False)
print(f'Datos guardados en {ruta_path + ruta_sabana_fabrica_full}')


df_sabana_fabrica_full.tail(10)

In [ ]:
df_sabana_fabrica_full[df_sabana_fabrica_full['AÑO'] == 2023].head(10)

Estadísticas importantes

In [ ]:
# Mostramos estadísticas importantes
# -------------
# -----------
# ---------

# --- Seleccionamos PLANTACION
# ---
df_sabana_fabrica_full_tendencia = df_sabana_fabrica_full[df_sabana_fabrica_full['PLANTACION'] == 'Palmawasi']
# ---

# Función personalizada para obtener el mínimo sin considerar ceros
def min_sin_ceros(x):
    x_no_cero = x[x != 0]
    if len(x_no_cero) == 0:
        return 0  # O algún valor representativo si todos son ceros
    return x_no_cero.min()

# Agrupar por la nueva columna 'AÑO_MES'
grouped_sabana_fabrica_full = df_sabana_fabrica_full_tendencia.groupby('AÑO_MES')

# Crear el DataFrame resumen
resumen_sabana_fabrica_full = grouped_sabana_fabrica_full.agg(
    nro_dias_con_rff_procesado=('RFF PROCESADO', lambda x: (x != 0).sum()),
    nro_dias_sin_rff_procesado=('RFF PROCESADO', lambda x: (x == 0).sum()),
    rff_procesado_suma=('RFF PROCESADO', 'sum'),
    rff_procesado_promedio=('RFF PROCESADO', 'mean'),
    mediana_rff_procesado=('RFF PROCESADO', 'median'),
    minimo_rff_procesado=('RFF PROCESADO', 'min'),
    minimo_rff_procesado_sin_ceros=('RFF PROCESADO', min_sin_ceros),
    maximo_rff_procesado=('RFF PROCESADO', 'max'),
    varianza_rff_procesado=('RFF PROCESADO', 'var'),
    desviacion_estandar_rff_procesado=('RFF PROCESADO', 'std')
).reset_index()

# Convertir 'AÑO_MES' al formato datetime
resumen_sabana_fabrica_full['AÑO_MES'] = pd.to_datetime(resumen_sabana_fabrica_full['AÑO_MES'], format='%Y-%m')
# Mostrar el DataFrame resumen
resumen_sabana_fabrica_full = resumen_sabana_fabrica_full.sort_values(by='AÑO_MES')
resumen_sabana_fabrica_full

In [ ]:
resumen_sabana_fabrica_full.dtypes

In [ ]:
# Crear el gráfico de líneas para 'toneladas_dia'
plt.figure(figsize=(15, 5))
plt.plot(resumen_sabana_fabrica_full['AÑO_MES'], resumen_sabana_fabrica_full['nro_dias_sin_rff_procesado'], marker='o', linestyle='-', color='green', label='RFF Procesado')
plt.title('RFF Procesado por Día a través del Tiempo')
plt.xlabel('Fecha')
plt.ylabel('RFF Procesado por Día')
plt.grid(True)
plt.legend()
plt.xticks(rotation=45)



Analizando tendencias

In [ ]:
# Filtramos la tendencia de interes
# -------------

# ---
df_sabana_fabrica_full_tendencia = df_sabana_fabrica_full[df_sabana_fabrica_full['PLANTACION'] == 'Palmawasi']
# ---

plantacion_name = df_sabana_fabrica_full_tendencia['PLANTACION'].unique()

# Crear el gráfico de líneas para 'toneladas_dia'
plt.figure(figsize=(15, 5))
plt.plot(df_sabana_fabrica_full_tendencia['FECHA'], df_sabana_fabrica_full_tendencia['RFF PROCESADO'], marker='o', linestyle='-', color='green', label='RFF Procesado')
plt.title('RFF Procesado por Día a través del Tiempo - '+ plantacion_name[0])
plt.xlabel('Fecha')
plt.ylabel('RFF Procesado por Día')
plt.grid(True)
plt.legend()
plt.xticks(rotation=45)

# Mostrar la gráfica
print(df_sabana_fabrica_full[df_sabana_fabrica_full['RFF PROCESADO'] == 0].head())
plt.tight_layout()
plt.show()